In [4]:
from django.shortcuts import render
from django.shortcuts import redirect
import urllib.parse
import requests


def meta_auth(request):
    params = {
        'client_id': 1152489759401130,
        'redirect_uri': 'http://localhost:8000/auth/meta/callback/',
        'scope': 'pages_manage_engagement,pages_read_engagement,instagram_manage_messages',
        'response_type': 'code',
    }
    url = 'https://www.facebook.com/v19.0/dialog/oauth?' + urllib.parse.urlencode(params)
    return redirect(url)



def meta_callback(request):
    code = request.GET.get('code')
    token_url = 'https://graph.facebook.com/v19.0/oauth/access_token'
    params = {
        'client_id': settings.META_APP_ID,
        'client_secret': settings.META_APP_SECRET,
        'redirect_uri': settings.META_REDIRECT_URI,
        'code': code,
    }
    response = requests.get(token_url, params=params).json()
    access_token = response['access_token']
    
    # Store the token securely (e.g., in the database)
    request.user.meta_access_token = access_token
    request.user.save()
    
    return redirect('dashboard')

def get_page_id(access_token):
    url = 'https://graph.facebook.com/v19.0/me/accounts'
    response = requests.get(url, params={'access_token': access_token}).json()
    #return response
    return response['data'][0]['id']  # First Page ID


def get_conversations(access_token, page_id):
    url = f'https://graph.facebook.com/v19.0/{page_id}/conversations'
    params = {
        'fields': 'messages{message,from,created_time}',
        'access_token': access_token,
    }
    response = requests.get(url, params=params).json()
    #return response['data']
    return response


def get_instagram_messages(access_token, page_id):
    # Get Instagram Business Account ID
    url = f'https://graph.facebook.com/v19.0/{page_id}'
    params = {
        'fields': 'instagram_business_account',
        'access_token': access_token,
    }
    ig_account_id = requests.get(url, params=params).json()['instagram_business_account']['id']

    # Fetch Instagram conversations
    url = f'https://graph.facebook.com/v19.0/{ig_account_id}/conversations'
    params = {
        'fields': 'messages{id,text,from,timestamp}',
        'access_token': access_token,
    }
    response = requests.get(url, params=params).json()
    return response['data']

access_token = 'EAAQYLu63ZAKoBOwZBsqKNzLKrKGaw2ZBRu069niMh8ZCnONtpYwsxCuASXfYuIJwYG0Gz07ZBzMlz010h6a10vMwpOSVQCcH3JOesR21M6GqrtpMQL7HyoZABOPRwrozDEHZCEXGVvMEQFMam85LQL5JCa1uZBIT5bjAPWZBagZCM3RkuTxP18p6ecxac1iFWG9e7K9CXdg13zxnLf4uiIcf09g0svBOrZBeUfGSRmA7dsq1zAtswhxxLU0'
pageid = '251068442394509'
page_ac = 'EAAQYLu63ZAKoBO24mQ7MUoVKmz9byOg5COWdaVJGMpELfpCcCxZAnc0fZCw76ZAdf0uOtaBP27pwPZBwZBA4g90NGhJojnfAkohqiRMgQFjEAJ9lOIUIZCr3fYouj363BzAprzk6njFzHKjXXTqT1RzYyDpNtPv3qnO7CDHsujcGYU7JwicfS6xn8YZAzrZC1Yd8ZD'

In [1]:
def get_page_access_token(user_access_token, page_id):
    url = f'https://graph.facebook.com/v19.0/{page_id}'
    params = {
        'fields': 'access_token',
        'access_token': user_access_token,
    }
    response = requests.get(url, params=params).json()
    return response['access_token']  # Page-specific token

def get_page_id(access_token):
    url = 'https://graph.facebook.com/v19.0/me/accounts'
    response = requests.get(url, params={'access_token': access_token}).json()
    
    if 'error' in response:
        raise Exception(f"API Error: {response['error']['message']}")
    
    if not response.get('data'):
        raise Exception("No Pages found. Ensure the user is a Page Admin.")
    
    return response['data'][0]['id']

def get_conversations(page_access_token, page_id):
    url = f'https://graph.facebook.com/v19.0/{page_id}/conversations'
    params = {
        'fields': 'messages{message,from,created_time}',
        'access_token': page_access_token,  # Use PAGE token here
    }
    response = requests.get(url, params=params).json()
    
    if 'error' in response:
        print("Graph API Error:", response['error'])
        return []
    
    return response.get('data', [])

In [6]:
pageid = get_page_id(access_token)

In [7]:
page_ac = get_page_access_token(access_token, pageid)

In [8]:
conversations = get_conversations(page_ac,pageid)

In [26]:
a = conversations['data'][3]

In [31]:
a['messages']['data']

[{'message': "Hi, thanks for contacting us. We've received your message and appreciate you reaching out.",
  'from': {'name': 'Scooter Shop',
   'email': '251068442394509@facebook.com',
   'id': '251068442394509'},
  'created_time': '2024-06-27T13:51:13+0000',
  'id': 'm_u7yiY6vciRMXg_6wqr6j5hVk-ZXlCoYo1BkxuBA2pV3twmF9uD4eWeRfJW8IbLM7Wax5Dp2eUFx_1o3egXbdKA'},
 {'message': 'გამარჯობა',
  'from': {'name': 'Gio Bareladze',
   'email': '9498358326890949@facebook.com',
   'id': '9498358326890949'},
  'created_time': '2024-06-27T13:51:12+0000',
  'id': 'm_5oBxTjEMC2kmmuldsYBvQBVk-ZXlCoYo1BkxuBA2pV3zhrVe5L43JxQcqwFRNq72BkIpoZYwjzimvh8-Xiezag'}]

In [ ]:
get_instagram_messages(page_ac,pageid)

KeyError: 'instagram_business_account'